In [9]:
import json
import numpy as np
import random
import nltk
from string import punctuation
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
stemmer = LancasterStemmer()

In [2]:
with open('Anthem_data.json') as json_data:
    json_data = json.load(json_data)

In [3]:
total_words = []
words = []
classes = ['Access_Service_Benefit', 'Appeals_and_Complaints', 'Benifit_Coverage', 'Claim', 'Eligibility', 
           'General_policy', 'Not_Covered', 'Payment', 'Termination_Nonrenewal_Continuation']

for class_name in classes:
    for word_list in json_data[class_name]:
        words.extend(word_list)

# stem and lower each word and remove duplicates
words = [stemmer.stem(w) for w in words]
words = sorted(list(set(words)))

In [4]:
# create our training data
training = []
output = []

# create an empty array for our output
output_empty = [0] * len(classes)

for class_name in classes:
    for word_list in json_data[class_name]:
        bag = []
        pattern_words = [stemmer.stem(word) for word in word_list]
        for w in words:
            bag.append(1) if w in pattern_words else bag.append(0)
        
        # output is a '0' for each tag and '1' for current tag
        output_row = list(output_empty)
        output_row[classes.index(class_name)] = 1
        training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [5]:
#Model structure
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
seed = 7
np.random.seed(seed)

#sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim = len(train_x[0]), activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(len(train_y[0]), activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
def model_train():
    model = create_model()
    model.summary()
    model.fit(np.array(train_x), np.array(train_y), batch_size=60, epochs=150, verbose=1)
    return model
trained_model = model_train()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 12)                23136     
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_3 (Dense)              (None, 9)                 81        
Total params: 23,321
Trainable params: 23,321
Non-trainable params: 0
_________________________________________________________________
Epoch 1/150
1694/1694 [==============================] - 1s 456us/step - loss: 2.1325 - acc: 0.3843
Epoch 2/150
1694/1694 [==============================] - 0s 32us/step - loss: 1.9770 - acc: 0.4144
Epoch 3/150
1694/1694 [==============================] - 0s 34us/step - loss: 1.8226 - acc: 0.4144
Epoch 4/150
1694/1694 [==============================] - 0s 33us/step - loss: 1.6942 - acc: 0.4162
Epoch

1694/1694 [==============================] - 0s 31us/step - loss: 0.0286 - acc: 0.9970
Epoch 76/150
1694/1694 [==============================] - 0s 31us/step - loss: 0.0275 - acc: 0.9982
Epoch 77/150
1694/1694 [==============================] - 0s 35us/step - loss: 0.0267 - acc: 0.9982
Epoch 78/150
1694/1694 [==============================] - 0s 30us/step - loss: 0.0259 - acc: 0.9976
Epoch 79/150
1694/1694 [==============================] - 0s 43us/step - loss: 0.0247 - acc: 0.9965
Epoch 80/150
1694/1694 [==============================] - 0s 41us/step - loss: 0.0238 - acc: 0.9982
Epoch 81/150
1694/1694 [==============================] - 0s 40us/step - loss: 0.0231 - acc: 0.9982
Epoch 82/150
1694/1694 [==============================] - 0s 40us/step - loss: 0.0226 - acc: 0.9976
Epoch 83/150
1694/1694 [==============================] - 0s 40us/step - loss: 0.0218 - acc: 0.9976
Epoch 84/150
1694/1694 [==============================] - 0s 40us/step - loss: 0.0209 - acc: 0.9976
Epoch 85/150


In [7]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words if word not in (list(punctuation) + list(stopwords.words('english')))]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return bag

In [10]:
query = ''
class_index = trained_model.predict(np.array(bow(query, words)).reshape(1,len(train_x[0])))[0].argmax()
print("Class it belongs to ",classes[class_index])

Class it belongs to  Benifit_Coverage
